In [6]:
import webvtt
from os import listdir
from os.path import isfile, join
import pandas as pd
from pydub import AudioSegment
import os
import shutil
import natsort 

In [2]:
if(False):
    i=0
    for caption in webvtt.read('/media/home/megha/5_wav2letter/WAV_2_LETTER/wav2letter/tutorials/Alugha_Eng/dev-clean-deu/00001/00001-1/Why_Can\'t_You_Fly%3F_deu.vtt'):
        print(caption)
        print(i)
        print('----------')
        print(caption.start) # start timestamp in text format
        print(caption.end) # end timestamp in text format
        print(caption.text) # caption text
        i=i+1

In [24]:
rootdir1 = "/media/home/megha/5_wav2letter/WAV_2_LETTER/wav2letter/tutorials/0_deutche_data/1_preprocessing/"
original_file_backup1 = "/media/home/megha/5_wav2letter/WAV_2_LETTER/wav2letter/tutorials/0_deutche_data/2_backup"




rootdir = "/media/home/megha/5_wav2letter/WAV_2_LETTER/wav2letter/tutorials/0_deutche_data/test"
original_file_backup = "/media/home/megha/5_wav2letter/WAV_2_LETTER/wav2letter/tutorials/0_deutche_data/backup"

In [25]:
source = "/media/home/megha/5_wav2letter/WAV_2_LETTER/wav2letter/tutorials/Alugha_Eng/test/SUB-145/12_Incredible_Facts_About_Russia_180414_mixdown_FTD12_facts_DEU.wav"
source1 = "/media/home/megha/5_wav2letter/WAV_2_LETTER/wav2letter/tutorials/Alugha_Eng/test/SUB-109/0154_RLL_Tornadoes_180329_mixdown_RLL_Tornadoes_deu.wav"
source2 = "/media/home/megha/5_wav2letter/WAV_2_LETTER/wav2letter/tutorials/Alugha_Eng/test/SUB-328/Do You Have Free Will_mixdown_ln_FreeWill_DEU.wav"
dest = "/media/home/megha/5_wav2letter/WAV_2_LETTER/wav2letter/tutorials/Alugha_Eng/test/SUB-145"
dest1= "/media/home/megha/5_wav2letter/WAV_2_LETTER/wav2letter/tutorials/Alugha_Eng/test/SUB-109"
dest2 = "/media/home/megha/5_wav2letter/WAV_2_LETTER/wav2letter/tutorials/Alugha_Eng/test/SUB-328"
name = "abcdef"
os.system('ffmpeg -i "{}" -acodec pcm_s16le -ac 1 -ar 16000 {}/{}.wav'.format(source2,dest2,name))# (source,target,name_of_file)

0

In [25]:
#1)
"""
converting all audio to wav file with the following specification. 
    -WAVE files with 16-bit
    -16 kHz
    -mono 
"""
for Dir in natsort.natsorted(os.listdir(rootdir)):
    for subdir in natsort.natsorted(os.listdir(rootdir+"/"+Dir)):
        print("----------------------"+subdir+"---------------------")
        length = len(os.listdir(rootdir+"/"+Dir+"/"+subdir))
        if(length > 2):
            print("file already converted")
        else:
            for file in os.listdir(rootdir+"/"+Dir+"/"+subdir): 
                if(file.endswith(".wav")):
                    actual_filename = file[:-4]+"_C"
                    a = os.system('ffmpeg -i "{}" -acodec pcm_s16le -ac 1 -ar 16000 "{}/{}".wav'.format(rootdir+"/"+Dir+"/"+subdir+"/"+file, rootdir+"/"+Dir+"/"+subdir, actual_filename))# (source,target(half path),name_of_file)
                    print(a)
                    if(a==256):
                        print("Did not get converted")
                        
#------------------------------------------------Backing up----------------------------------------------------------

for Dir in natsort.natsorted(os.listdir(rootdir)):
    for subdir in os.listdir(rootdir+"/"+Dir):
        print("----------------------"+subdir+"---------------------")
        length = len(os.listdir(rootdir+"/"+Dir+"/"+subdir))
        if(length > 2):
            #move the file
            for file in os.listdir(rootdir+"/"+Dir+"/"+subdir):
                if(file.endswith(".wav") and file.split('_')[-1]!="C.wav"):
                    shutil.move(rootdir+"/"+Dir+"/"+subdir+"/"+file, original_file_backup) # (source,target)
        if(length == 2):
            print(subdir+ " is being removed")
            #move complete folder the the backup folder
            shutil.move(rootdir+"/"+Dir+"/"+subdir, original_file_backup) # (source,target)
            

----------------------SUB-296---------------------
0
----------------------SUB-296---------------------


In [26]:
# 2)
"""
creating chunks and saving in the same folder
"""
for Dir in os.listdir(rootdir):
    for subdir in os.listdir(rootdir+"/"+Dir):
        print("1. -----------------------"+subdir)
        df  = pd.DataFrame(columns = ['A', 'B', 'C'])
        for file in os.listdir(rootdir+"/"+Dir+"/"+subdir):
            if(file.endswith(".vtt")):                
                for caption in webvtt.read(rootdir+"/"+Dir+"/"+subdir+"/"+file):
                    df2 = pd.DataFrame([[caption.start,caption.end,caption.text]], columns=list('ABC'))
                    df = df.append(df2)
                df = df.rename(index=str, columns={"A": "start", "B": "end","C" : "text"})
        for file in os.listdir(rootdir+"/"+Dir+"/"+subdir):
            if(file.endswith(".wav") and file.split('_')[-1]=="C.wav"):                
                myaudio = AudioSegment.from_file(rootdir+"/"+Dir+"/"+subdir+"/"+file, "wav")
                for i in range(0, len(df)):
                    start_time = df['start'].iloc[i]
                    end_time = df['end'].iloc[i]
                    # -----------------------------START-TIME-----------------------------------
                    s_hours, s_minutes, s_seconds = (["0", "0"] + start_time.split(":"))[-3:]
                    s_hours = int(s_hours)
                    s_minutes = int(s_minutes)
                    s_seconds = float(s_seconds)
                    s_millisecond = int(3600000 * s_hours + 60000 * s_minutes + 1000 * s_seconds)
                    START = s_millisecond
                    #-------------------------------END-TIME---------------------------------------
                    e_hours, e_minutes, e_seconds = (["0", "0"] + end_time.split(":"))[-3:]
                    e_hours = int(e_hours)
                    e_minutes = int(e_minutes)
                    e_seconds = float(e_seconds)
                    e_miliseconds = int(3600000 * e_hours + 60000 * e_minutes + 1000 * e_seconds)
                    END = e_miliseconds
                    #--------------------------------------------------------------------------
                    chunk = myaudio[START:END]
                    chunk_name = file.split('.')[-2]+"_"+str(i)+".flac"
                    if not os.path.exists(rootdir+"/"+Dir+"/"+subdir+"/"+chunk_name):
                        print("exporting", chunk_name)
                        chunk.export(rootdir+"/"+Dir+"/"+subdir+"/"+chunk_name, format="flac")
                if not os.path.exists(original_file_backup+"/"+file):
                    shutil.move(rootdir+"/"+Dir+"/"+subdir+"/"+file, original_file_backup) # (source,target)
                    print("Backing up")
                else:
                    os.remove(rootdir+"/"+Dir+"/"+subdir+"/"+file)
                    print("File already exist. So just removing")
#----------------------------------------------txt-----------------------------------------------------------------            
        txt_name = ""        
        for file in natsort.natsorted(os.listdir(rootdir+"/"+Dir+"/"+subdir)):
            if(file.endswith(".vtt")):
                txt_name = file.split(".")[-2]
        with open(rootdir+"/"+Dir+"/"+subdir+"/"+txt_name+"_C.trans.txt", "w") as txt_file:
            i=0
            for file in natsort.natsorted(os.listdir(rootdir+"/"+Dir+"/"+subdir)):                
                if(file.endswith(".wav")):
                    wav_file_name = file.split(".")[-2]
                    #print(df['text'].iloc[i])
                    txt_file.write(wav_file_name+" "+df['text'].iloc[i].upper()+"\n")
                    i = i+1
        for file in natsort.natsorted(os.listdir(rootdir+"/"+Dir+"/"+subdir)):
            if(file.endswith(".vtt")):
                if not os.path.exists(original_file_backup+"/"+file):
                    shutil.move(rootdir+"/"+Dir+"/"+subdir+"/"+file, original_file_backup) # (source,target)
                else:
                    os.remove(rootdir+"/"+Dir+"/"+subdir+"/"+file)
                    print("File already exist. So just removing")
               
               
            

1. -----------------------SUB-296
exporting 0165_DEU_C_0.flac
exporting 0165_DEU_C_1.flac
exporting 0165_DEU_C_2.flac
exporting 0165_DEU_C_3.flac
exporting 0165_DEU_C_4.flac
exporting 0165_DEU_C_5.flac
exporting 0165_DEU_C_6.flac
exporting 0165_DEU_C_7.flac
exporting 0165_DEU_C_8.flac
exporting 0165_DEU_C_9.flac
exporting 0165_DEU_C_10.flac
exporting 0165_DEU_C_11.flac
exporting 0165_DEU_C_12.flac
exporting 0165_DEU_C_13.flac
exporting 0165_DEU_C_14.flac
exporting 0165_DEU_C_15.flac
exporting 0165_DEU_C_16.flac
exporting 0165_DEU_C_17.flac
exporting 0165_DEU_C_18.flac
exporting 0165_DEU_C_19.flac
exporting 0165_DEU_C_20.flac
exporting 0165_DEU_C_21.flac
exporting 0165_DEU_C_22.flac
exporting 0165_DEU_C_23.flac
exporting 0165_DEU_C_24.flac
exporting 0165_DEU_C_25.flac
exporting 0165_DEU_C_26.flac
exporting 0165_DEU_C_27.flac
exporting 0165_DEU_C_28.flac
exporting 0165_DEU_C_29.flac
exporting 0165_DEU_C_30.flac
exporting 0165_DEU_C_31.flac
exporting 0165_DEU_C_32.flac
exporting 0165_DEU_

In [21]:
SAVE_DIR = "/media/home/megha/5_wav2letter/WAV_2_LETTER/wav2letter/tutorials/Alugha_Eng/dev-clean-deu/"
dirName = "Why Can't You Fly_mixdown_ln_cantFly_DEU" # direct file name as directory name
dirPath = SAVE_DIR+"/"+dirName
#here you put a loop to read all the wav files
myaudio = AudioSegment.from_file("/media/home/megha/5_wav2letter/WAV_2_LETTER/wav2letter/tutorials/Alugha_Eng/dev-clean-deu/00001/00001-1/Why Can't You Fly_mixdown_ln_cantFly_DEU.wav", "wav")
for i in range(0, len(df)):
    if not os.path.exists(dirPath):
        os.makedirs(dirPath)    
    start_time = df['start'].iloc[i]
    end_time = df['end'].iloc[i]    
    # ------------------------------------------------------------------------
    s_hours, s_minutes, s_seconds = (["0", "0"] + start_time.split(":"))[-3:]
    s_hours = int(s_hours)
    s_minutes = int(s_minutes)
    s_seconds = float(s_seconds)
    s_millisecond = int(3600000 * s_hours + 60000 * s_minutes + 1000 * s_seconds)
    START = s_millisecond
    #--------------------------------------------------------------------------
    e_hours, e_minutes, e_seconds = (["0", "0"] + end_time.split(":"))[-3:]
    e_hours = int(e_hours)
    e_minutes = int(e_minutes)
    e_seconds = float(e_seconds)
    e_miliseconds = int(3600000 * e_hours + 60000 * e_minutes + 1000 * e_seconds)
    END = e_miliseconds
    #--------------------------------------------------------------------------
    chunk = myaudio[START:END]    
    chunk_name =  str(i)+".wav" 
    file_destination = dirPath+"/"+chunk_name
    if not os.path.exists(file_destination):
        print("exporting", chunk_name)
        chunk.export(file_destination , format="wav")



exporting 0.wav
exporting 1.wav
exporting 2.wav
exporting 3.wav
exporting 4.wav
exporting 5.wav
exporting 6.wav
exporting 7.wav
exporting 8.wav
exporting 9.wav
exporting 10.wav
exporting 11.wav
exporting 12.wav
exporting 13.wav
exporting 14.wav
exporting 15.wav
exporting 16.wav
exporting 17.wav
exporting 18.wav
exporting 19.wav
exporting 20.wav
exporting 21.wav
exporting 22.wav
exporting 23.wav
